In [42]:
import  dask.dataframe as dd

# Carregar os dados
# Read the Parquet files with Dask
train_ddf = dd.read_parquet('train.parquet')
test_ddf = dd.read_parquet('test.parquet')

# pegando sample dos dados
train_df = train_ddf.sample(frac=0.02, random_state=42).persist()
test_df = test_ddf.sample(frac=0.02, random_state=42).persist()



In [43]:
train_df.shape, test_df.shape

((Delayed('int-651612d1-9a19-4213-af36-b8795a89a4a6'), 3),
 (Delayed('int-a8f90c8a-c7d6-4468-bd27-f4041dc2dc1a'), 3))

In [44]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, BatchNormalization, Bidirectional

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report

import gc
from dask.diagnostics import ProgressBar
from tensorflow.keras.optimizers import Adam, RMSprop, SGD, Adagrad
from tensorflow.keras import regularizers
import tensorflow as tf     

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
import unidecode
# Downloads necessários para o NLTK
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Lucia\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Lucia\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lucia\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

##### Tokenização e Padronização para LSTM  

In [45]:
# Função para preprocessar texto
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def preprocess_text_traditional(text):
    text = unidecode.unidecode(text)  # Remove accents
    text = text.lower()  # Convert to lowercase
    text = text.translate(str.maketrans('', '', string.punctuation))  # Remove punctuation
    words = word_tokenize(text)  # Tokenize
    words = [word for word in words if word not in stop_words]  # Remove stopwords
    words = [lemmatizer.lemmatize(word) for word in words]  # Lemmatize
    return ' '.join(words)  # Join tokens back into a string

def preprocess_texts(df):
    df['processed_text'] = df['review_text'].apply(preprocess_text_traditional, meta=('review_text', 'str'))
    return df

In [46]:

# Apply preprocessing in parallel
with ProgressBar():
    train_df = preprocess_texts(train_df).persist()
    test_df = preprocess_texts(test_df).persist()

# Compute the preprocessed text columns    
X_train_ddf = train_df[['processed_text']].compute()
y_train_ddf = train_df['rating'].compute()
X_test_ddf = test_df[['processed_text']].compute()
y_test_ddf = test_df['rating'].compute()

# Extract processed text columns
X_train = X_train_ddf['processed_text']
X_test = X_test_ddf['processed_text']

# Garbage collection to free memory
gc.collect()

[########################################] | 100% Completed | 29.80 s
[########################################] | 100% Completed | 3.21 ss


86090

In [47]:
vocab_size = 5000  # Use the num_words value directly
# Initialize tokenizer specific for LSTM
tokenizer_lstm = Tokenizer(num_words=vocab_size)

# Tokenization and Padding
tokenizer_lstm.fit_on_texts(X_train)

X_train_seq = tokenizer_lstm.texts_to_sequences(X_train)
X_train_pad = pad_sequences(X_train_seq, maxlen=100)

X_test_seq = tokenizer_lstm.texts_to_sequences(X_test)
X_test_pad = pad_sequences(X_test_seq, maxlen=100)

# Encode target labels to one-hot
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train_ddf)
y_train = tf.keras.utils.to_categorical(y_train, num_classes=2)

y_test = label_encoder.transform(y_test_ddf)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=2)

##### Função para criar modelo LSTM 

In [66]:
# Função para criar modelo LSTM
def create_model(dropout_rate= 0.5, LSTM_layers= 1, LSTM_neurons= 64, dense_layers= 1, dense_neurons= 32, activation= 'relu'):
    try:
        
        model = Sequential()
        model.add(Embedding(input_dim=vocab_size, output_dim=128, input_shape=(100,)))
        
        
        if LSTM_layers == 3:
            model.add(Bidirectional(LSTM(LSTM_neurons, dropout=dropout_rate, recurrent_dropout=dropout_rate, return_sequences=True)))
            model.add(BatchNormalization())
            model.add(Bidirectional(LSTM(LSTM_neurons, dropout=dropout_rate, recurrent_dropout=dropout_rate, return_sequences=True)))
            model.add(BatchNormalization())
        
        if LSTM_layers == 2:
            model.add(Bidirectional(LSTM(LSTM_neurons, dropout=dropout_rate, recurrent_dropout=dropout_rate, return_sequences=True)))
            model.add(BatchNormalization())
            
        
        model.add(Bidirectional(LSTM(LSTM_neurons, dropout=dropout_rate, recurrent_dropout=dropout_rate, return_sequences=False)))
        model.add(BatchNormalization())
        
        for _ in range(dense_layers-1):
            model.add(Dense(dense_neurons, activation=activation, kernel_regularizer=regularizers.l2(0.01)))
            model.add(BatchNormalization())
            model.add(Dropout(dropout_rate))
        
        model.add(Dense(dense_neurons, activation=activation, kernel_regularizer=regularizers.l2(0.01)))
        model.add(Dense(2, kernel_initializer='uniform', activation='softmax'))  # Adjusted this line
        return model
    except Exception as e:
        print(f"An error occurred while adding layers: {e}")
        return None

#### Custom Callback   

Criamos um custom callback para monitorar o treinamento do modelo e salvar os melhores pesos. De forma que eu posso escolher quantas épocas esperar para reduzir o learning rate e quantas épocas esperar para restaurar os melhores pesos. Então Criei uma Rotina dessa forma:

- Se o modelo não melhorar por um número de épocas (patience) ele restaura os melhores pesos.
- Se o modelo não melhorar por um número de épocas (lr_patience) ele reduz o learning rate. 
- Se o learning rate chegar no mínimo, ele para o treinamento.  

Dessa forma a estrutura deixa o modelo re-testar o learning_rate no melhor ponto de mínimo local, e não deixa o modelo divagar.
se mesmo assim aquele learning_rate não funcionar, ele vai diminuindo até chegar no mínimo e parar o treinamento. 

Também criei outra rotina onde ela:
- salva os pesos e os resultados de 3 épocas que rodam com os mesmo pesos
- compara se uma dessas 3 épocas melhorou o modelo.
- Se Melhorou atualiza os pesos
- Se piorou, ele reduz o learning rate e realiza mais 3 épocas
- Se o learning_rate chegar no mínimo encerra-se o treinamento.

Infelizmente o Callback ReduceLROnPlateau não funciona dessa forma, ele reduz o learning rate mas ele aplica nos pesos atuais que podem ter divagado muito longe, e não nos melhores pesos. então por exemplo, se com o learning rate de 0.01 o modelo piorar significativamente no periodo de pacience, ele reduzirá o learning rate para 0.004, mas continuará treinando com os pesos atuais que pioraram o modelo e estão em um minimo local pior. Por exemplo: ele no inicio do treinamento encontrou um mínimo local de val_loss de 0.4 7 epócas dentro do treinamento, mas nas outras interações ele divagou para 0.55, o ReduceLROnPlateau vai reduzir o learning rate para 0.001 mas o modelo vai continuar do local val_loss de 0.55 e com um learning_rate menor dificilmente irá achar um mínimo local melhor.

##### Custom Callback   

Criamos um custom callback para monitorar o treinamento do modelo e salvar os melhores pesos. De forma que eu posso escolher quantas épocas esperar para reduzir o learning rate e quantas épocas esperar para restaurar os melhores pesos. Então Criei uma Rotina dessa forma:

- Se o modelo não melhorar por um número de épocas (patience) ele restaura os melhores pesos.
- Se o modelo não melhorar por um número de épocas (lr_patience) ele reduz o learning rate. 
- Se o learning rate chegar no mínimo, ele para o treinamento.  

Dessa forma a estrutura deixa o modelo re-testar o learning_rate no melhor ponto de mínimo local, e não deixa o modelo divagar.
se mesmo assim aquele learning_rate não funcionar, ele vai diminuindo até chegar no mínimo e parar o treinamento. 

Também criei outra rotina onde ela:
- salva os pesos e os resultados de 3 épocas que rodam com os mesmo pesos
- compara se uma dessas 3 épocas melhorou o modelo.
- Se Melhorou atualiza os pesos
- Se piorou, ele reduz o learning rate e realiza mais 3 épocas
- Se o learning_rate chegar no mínimo encerra-se o treinamento.

Infelizmente o Callback ReduceLROnPlateau não funciona dessa forma, ele reduz o learning rate mas ele aplica nos pesos atuais que podem ter divagado muito longe, e não nos melhores pesos. então por exemplo, se com o learning rate de 0.01 o modelo piorar significativamente no periodo de pacience, ele reduzirá o learning rate para 0.004, mas continuará treinando com os pesos atuais que pioraram o modelo e estão em um minimo local pior. Por exemplo: ele no inicio do treinamento encontrou um mínimo local de val_loss de 0.4 7 epócas dentro do treinamento, mas nas outras interações ele divagou para 0.55, o ReduceLROnPlateau vai reduzir o learning rate para 0.001 mas o modelo vai continuar do local val_loss de 0.55 e com um learning_rate menor dificilmente irá achar um mínimo local melhor.

In [67]:
from tensorflow.keras.callbacks import Callback
import numpy as np

class CustomCallback(Callback):
    def __init__(self, patience=0, lr_patience=3, reduce_factor=0.5, min_lr=0.0001):
        super(CustomCallback, self).__init__()
        self.patience = patience
        self.lr_patience = lr_patience 
        self.wait = 0
        self.lr_wait = 0
        self.best_weights = None
        self.best_loss = np.inf
        self.reduce_factor = reduce_factor
        self.min_lr = min_lr
        self.best_epoch = 0
        self.results = []
        self.weights = []

    def on_train_begin(self, logs=None):
        self.wait = 0
        self.lr_wait = 0
        self.best_weights = None
        self.best_loss = np.inf
        self.results = []
        self.weights = []

    def on_epoch_begin(self, epoch, logs=None):
        lr = float(tf.keras.backend.get_value(self.model.optimizer.learning_rate))
        print("\nLearning rate is %6.4f." % (lr))
        print(f"Epoch {epoch}:")
        # print("current wait count:", self.wait)
        print("best loss:", self.best_loss)
        print("current loss:", logs.get("val_loss"))
        if self.best_weights is not None:
            self.model.set_weights(self.best_weights)
    
    def on_epoch_end(self, epoch, logs=None):
        current_loss = logs.get("val_loss")
        self.results.append(current_loss)
        self.weights.append(self.model.get_weights())
        if self.best_weights is None:
            self.best_weights = self.model.get_weights()  
        if len(self.results) == 3:
            min_loss_index = np.argmin(self.results)
            min_loss = self.results[min_loss_index]
            if min_loss < self.best_loss:
                self.best_loss = min_loss
                self.best_weights = self.weights[min_loss_index]
            else:
                old_lr = float(tf.keras.backend.get_value(self.model.optimizer.learning_rate))
                new_lr = old_lr * self.reduce_factor
                new_lr = max(new_lr, self.min_lr)
                if new_lr == self.min_lr:
                    self.model.stop_training = True
                self.model.optimizer.learning_rate = tf.Variable(new_lr)
            self.results = []
            self.weights = []
    
    # def on_epoch_end(self, epoch, logs=None):
    #     current_loss = logs.get("val_loss")
    #     if np.less(current_loss, self.best_loss):
    #         self.best_loss = current_loss
    #         self.best_weights = self.model.get_weights()
    #         self.best_epoch = epoch
    #         self.wait = 0
    #         self.lr_wait = 0
    #     else:
    #         self.wait += 1
    #         self.lr_wait += 1
    #         print(f"\nLoss did not improve. Current wait count: {self.wait}.")
    #         if self.wait >= self.patience:
    #             print('restoring best weights') 
    #             self.model.set_weights(self.best_weights)
    #             self.wait = 0
    #         if self.lr_wait >= self.lr_patience:
    #             print('reducing learning rate')
    #             old_lr = float(tf.keras.backend.get_value(self.model.optimizer.learning_rate))
    #             new_lr = old_lr * self.reduce_factor
    #             new_lr = max(new_lr, self.min_lr)
    #             if new_lr == self.min_lr:
    #                 self.model.stop_training = True
    #             self.model.optimizer.learning_rate = tf.Variable(new_lr)
    #             self.lr_wait = 0
                
                
# Instantiate the custom callback and use it as an argument in model.fit
custom_callback = CustomCallback(patience=3,lr_patience=6)

##### Treinamento com os melhores parametros encontrados na random search mas com um número maior de épocas.

In [68]:
# Criar modelo com os melhores parametros
model_lstm = create_model(dropout_rate=0.5, LSTM_layers=2, LSTM_neurons=64, dense_layers=1, dense_neurons=32, activation='relu')
optimizer = RMSprop(learning_rate=0.001)
model_lstm.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# # Define callbacks
# early_stopping = EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True)
# reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.4, patience=1, min_lr=0.0001)
# model_checkpoint = ModelCheckpoint('best_model_lstm_checkpoint.keras', monitor='val_loss', save_best_only=True, save_weights_only=False)

C:\Users\Lucia\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [62]:

model_lstm.summary()

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_8 (Embedding)         │ (None, 100, 128)       │       640,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_16                │ (None, 100, 128)       │        98,816 │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_16          │ (None, 100, 128)       │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_17                │ (None, 128)            │        98,816 │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_17          │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 2)              │            66 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 842,850 (3.22 MB)

 Trainable params: 842,338 (3.21 MB)

 Non-trainable params: 512 (2.00 KB)

In [ ]:
model_lstm.fit(X_train_pad, y_train,
               batch_size=64, epochs=50, validation_data=(X_test_pad, y_test), verbose=2, callbacks=[custom_callback])

# Restaurar melhores pesos
model_lstm.set_weights(custom_callback.best_weights)

# salvar modelo
model_lstm.save('best_model_lstm_30_epochs.keras')

# Avaliar modelo    
y_pred = np.argmax(model_lstm.predict(X_test_pad), axis=-1)
y_test_binary = np.argmax(y_test, axis=-1)  # Convert y_test to binary format
accuracy_lstm = accuracy_score(y_test_binary, y_pred)  # Use y_test_binary here
# classification report
report_lstm = classification_report(y_test_binary, y_pred, zero_division= 0)  # Use y_test_binary here


In [28]:
print('Accuracy:', accuracy_lstm)       
print(report_lstm)


Accuracy: 0.8555
              precision    recall  f1-score   support

           0       0.86      0.85      0.86      4025
           1       0.85      0.86      0.85      3975

    accuracy                           0.86      8000
   macro avg       0.86      0.86      0.86      8000
weighted avg       0.86      0.86      0.86      8000

